In [146]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import re
import string
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

In [147]:

data = pd.read_excel('Superstorecleaned.xlsx')

In [148]:
# data = '#(Hello! I need to remove $pl @hars in Pyth@n)'     cleaned_data = data.translate(str.maketrans('','',string.punctuation))     print(cleaned_data)

In [163]:
def remove_special(x):
  s = x.translate(str.maketrans('','',string.punctuation))
  s = re.sub('[0-9]', '', s)
  return s

In [164]:
data['Product Name'] = data['Product Name'].apply(lambda x: remove_special(x)) 
data["Product Name"]

0                           Ikea Library with Doors Mobile
1                                  Acme Scissors Easy Grip
2                              Epson Receipt Printer White
3                           Rubbermaid Door Stop Erganomic
4        Cameo Interoffice Envelope with clear poly window
                               ...                        
51285                       Sanford Highlighters EasyErase
51286                  Avery Color Coded Labels  Label Set
51287                 Sanford Pencil Sharpener Fluorescent
51288                       Panasonic Card Printer Durable
51289                  Wilson Jones Binder Covers Recycled
Name: Product Name, Length: 51290, dtype: object

In [213]:
data[['Product Name', 'Customer ID', 'Quantity']][225:]

,Product Name,Customer ID,Quantity
225,Cuisinart Coffee Grinder Black,23403,1
226,Green Bar Computer Printout Paper Multicolor,27903,1
227,Cardinal Binding Machine Recycled,23403,1


In [167]:
# data['Quantity'].value_counts()

SyntaxError: ignored

In [168]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import re

In [169]:
data.columns

Index(['Row ID', 'Order ID', 'Ship Mode', 'Customer ID', 'Customer Name',
       'Segment', 'Product ID', 'Product Name', 'Sub-Category', 'Category',
       'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost',
       'Order Priority', 'City', 'Region', 'Order_Date', 'Ship_Date',
       'Postal Code'],
      dtype='object')

In [170]:
# lst = data['Product Name'].value_counts()[:50].index
# lst = lst.to_list()
# lst
# data.shape

In [171]:
# def convert_list(x):
#   for i in lst:
#     if x == i:
#       return x
#   return 0

In [172]:
# data['Product Name2'] = data['Product Name'].apply(lambda x: convert_list(x))
# # data['Product Name2'].value_counts()
# data = data[data['Product Name2'] != 0]
# data.shape

In [193]:
data = data.head(228)

In [194]:
# data['Sales'].apply(lambda x: convert_list(x))

In [195]:
data.shape

(228, 21)

In [196]:
data_filtering=data['Customer ID'].tolist()

In [197]:
emptyList =[]
for i in data_filtering:
    emptyList.append(re.findall('\d+', i))
    

In [198]:
df = pd.DataFrame(emptyList, columns=['Customer_ID'])

In [199]:
list_data=df['Customer_ID'].tolist()

In [200]:
data['Customer ID']=list_data

In [201]:
# data[['Product Name', 'Customer ID', 'Quantity']][225:]

In [202]:
# data = data.head(10000)

In [203]:
data.shape

(228, 21)

In [204]:
data['Product Name'] = data['Product Name'].str.strip()
data.dropna(axis=0, subset=['Customer ID'], inplace=True)
data['Customer ID'] = data['Customer ID'].astype('str')

In [205]:
basket = (data[data['Region'] =="South"]
          .groupby(['Customer ID', 'Product Name'])['Quantity']
         .sum().unstack().reset_index().fillna(0)
          .set_index('Customer ID'))

In [206]:
basket.shape

(14, 22)

In [207]:
basket

Product Name,Acme Scissors Easy Grip,Cameo Interoffice Envelope with clear poly window,Cisco Speaker Phone with Caller ID,Dania Library with Doors Metal,Eldon Lockers Wire Frame,Fellowes Folders Wire Frame,GlobeWeis Peel and Seal SecurityTint,Green Bar Parchment Paper Premium,Harbour Creations Shipping Labels Alphabetical,Hewlett Personal Copier HighSpeed,Ikea Library with Doors Mobile,KitchenAid Blender White,Nokia Smart Phone with Caller ID,Nokia Speaker Phone Cordless,Novimex Legal Exhibit Labels Alphabetical,Rubbermaid Door Stop Erganomic,SanDisk Computer Printout Paper x,SanDisk Note Cards x,Sauder Stackable Bookrack Mobile,Stanley Sketch Pad Blue,StarTech Phone Red,Wilson Jones Hole Reinforcements Recycled
Customer ID,,,,,,,,,,,,,,,,,,,,,,
110351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115053,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116051,9.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2103,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0
31803,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4203,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
79803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [208]:
basket.columns

Index(['Acme Scissors Easy Grip',
       'Cameo Interoffice Envelope with clear poly window',
       'Cisco Speaker Phone with Caller ID', 'Dania Library with Doors Metal',
       'Eldon Lockers Wire Frame', 'Fellowes Folders Wire Frame',
       'GlobeWeis Peel and Seal SecurityTint',
       'Green Bar Parchment Paper Premium',
       'Harbour Creations Shipping Labels Alphabetical',
       'Hewlett Personal Copier HighSpeed', 'Ikea Library with Doors Mobile',
       'KitchenAid Blender White', 'Nokia Smart Phone with Caller ID',
       'Nokia Speaker Phone Cordless',
       'Novimex Legal Exhibit Labels Alphabetical',
       'Rubbermaid Door Stop Erganomic', 'SanDisk Computer Printout Paper  x',
       'SanDisk Note Cards  x', 'Sauder Stackable Bookrack Mobile',
       'Stanley Sketch Pad Blue', 'StarTech Phone Red',
       'Wilson Jones Hole Reinforcements Recycled'],
      dtype='object', name='Product Name')

In [209]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)


In [210]:
basket_sets

Product Name,Acme Scissors Easy Grip,Cameo Interoffice Envelope with clear poly window,Cisco Speaker Phone with Caller ID,Dania Library with Doors Metal,Eldon Lockers Wire Frame,Fellowes Folders Wire Frame,GlobeWeis Peel and Seal SecurityTint,Green Bar Parchment Paper Premium,Harbour Creations Shipping Labels Alphabetical,Hewlett Personal Copier HighSpeed,Ikea Library with Doors Mobile,KitchenAid Blender White,Nokia Smart Phone with Caller ID,Nokia Speaker Phone Cordless,Novimex Legal Exhibit Labels Alphabetical,Rubbermaid Door Stop Erganomic,SanDisk Computer Printout Paper x,SanDisk Note Cards x,Sauder Stackable Bookrack Mobile,Stanley Sketch Pad Blue,StarTech Phone Red,Wilson Jones Hole Reinforcements Recycled
Customer ID,,,,,,,,,,,,,,,,,,,,,,
110351,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
115053,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
116051,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
120551,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
2103,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29403,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
31803,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4203,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
79803,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [211]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [212]:
#error at index 229
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cameo Interoffice Envelope with clear poly wi...,(Acme Scissors Easy Grip),0.071429,0.071429,0.071429,1.0,14.0,0.066327,inf
1,(Acme Scissors Easy Grip),(Cameo Interoffice Envelope with clear poly wi...,0.071429,0.071429,0.071429,1.0,14.0,0.066327,inf
2,(SanDisk Computer Printout Paper x),(Eldon Lockers Wire Frame),0.071429,0.071429,0.071429,1.0,14.0,0.066327,inf
3,(Eldon Lockers Wire Frame),(SanDisk Computer Printout Paper x),0.071429,0.071429,0.071429,1.0,14.0,0.066327,inf
4,(Stanley Sketch Pad Blue),(GlobeWeis Peel and Seal SecurityTint),0.071429,0.071429,0.071429,1.0,14.0,0.066327,inf
